# Capstone Project

## This notebook is the 3rd part of Assignment 2 in the Capstone Project Course

##### The pupose of this notebook is to collect data about neighborhood from the wikipedia page. Then store the Postal code, Borough, Neighborhood Name in a dataframe

In [1]:
#importing libraries

import pandas as pd
import numpy as np
import folium
import requests
from sklearn.cluster import KMeans 
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [2]:
#installing folium
#!pip -q install folium

In [3]:
#!conda install -c conda-forge geopy --yes
#!pip install geocoder


In [4]:
#!pip install pgeocode

In [ ]:
#library for fetching the longitude and the lattitude of a given address.

from geopy.geocoders import Nominatim
import geocoder
import pgeocode

In [ ]:
#Reading the table from the webpage

table=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=table[0]
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [ ]:
# Cleaning the data by removing all the borough with value as Not assigned and changing the neighbourhood to borough if it was Not assigned

rows_list=[]

for index,row in df.iterrows():
    dict1={}
    pc=row['Postal Code']
    bo=row['Borough']
    ne=row['Neighbourhood']
    #print("PC",pc ,bo,ne )
    if bo == 'Not assigned':
        continue
    else:
        if ne == "Not assigned":
            final_ne=bo
        else:
            final_ne=ne
        
    dict1['Postal Code']=pc
    dict1['Borough']=bo
    dict1['Neighborhood']=ne
    
    rows_list.append(dict1)
    
   # df1.append([pc,bo,final_ne])   
  #  print(df1)
    
df1=pd.DataFrame(rows_list)
    
        

In [ ]:
df1=df1[['Postal Code','Borough','Neighborhood']]
df1.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
df1.shape

(103, 3)

### On using the "geocode" package the longitude and latitude for some reason wasn't returned (and also I think I exhausted the daily limit by trying again and again) so I used another library called "pgeocode" that does the same work as "geocode".

In [ ]:
def get_geocoder(post):

    nomi = pgeocode.Nominatim('ca')
    x=nomi.query_postal_code('{}'.format(post))
    
    lat=x.latitude
    long=x.longitude
    #print(lat)
               
    return lat,long                        

df1['Latitude'], df1['Longitude'] = zip(*df1['Postal Code'].apply(get_geocoder))
df1=df1[['Postal Code','Borough','Neighborhood','Latitude','Longitude']]

df1.head()


### Selecting Boroughs that contain 'Toronto' in them.

In [ ]:
df_toronto=pd.DataFrame(columns=['Postal Code','Borough','Neighborhood','Latitude','Longitude'])

for i,rows in df1.iterrows():
    if 'Toronto' in rows['Borough']:
        #print("here")
        po=rows['Postal Code']
        bo=rows['Borough']
        nei=rows['Neighborhood']
        lat=rows['Latitude']
        long=rows['Longitude']
        #print(bo)    
        df_toronto = df_toronto.append({
            'Postal Code':po,
            'Borough':bo,
            'Neighborhood':nei,
            'Latitude':lat,
            'Longitude':long},ignore_index=True)
    
df_toronto

### Creating a map of  all the neighborhoods in Toronto

In [ ]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Getting the FourSquare credentials

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
t_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

In [ ]:
print(t_venues.shape)
t_venues.head()

In [ ]:
t_venues.groupby('Neighborhood').count()

### Determining unique venue categories

In [ ]:
print('There are {} uniques categories.'.format(len(t_venues['Venue Category'].unique())))

### Analyzing each neighborhood

In [ ]:
# one hot encoding
t_onehot = pd.get_dummies(t_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#t_onehot['Neighborhood'] = t_venues['Neighborhood'] 
t_onehot.pop('Neighborhood')
t_onehot.insert(0, 'Neighborhood', t_venues['Neighborhood'])

# move neighborhood column to the first column
#fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
#t_onehot = t_onehot[fixed_columns]

t_onehot.head()

In [ ]:
t_onehot.shape

### Grouping the dataset based on "Neighborhood" and calculating the mean for each venue category for that respective group

In [ ]:
t_grouped = t_onehot.groupby('Neighborhood').mean().reset_index()
t_grouped.head(10)

In [ ]:
t_grouped.shape

### Top 5 venues of each neighborhood

In [ ]:
num_top_venues = 5

for hood in t_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = t_grouped[t_grouped['Neighborhood'] == hood].T.reset_index()

    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]

    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Function to sort the venues in descending order

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Creating a new dataframe to show the top 10 venues of  neighbourhood

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = t_grouped['Neighborhood']

for ind in np.arange(t_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Run K-Means to cluster the neighborhoods into 6 clusters

In [ ]:
# set number of clusters
kclusters = 6

t_grouped_clustering = t_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

### New datframe that contains cluseter as well the 10 most common venue

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.pop('Cluster Labels')
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

t_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
t_merged = t_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

t_merged.head() # *beginning

### Visualizing the results

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_merged['Latitude'], t_merged['Longitude'], t_merged['Neighborhood'], t_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters